# Lab 1-2: Part 2

In [241]:
# Import libraries

import arcpy
import psycopg2
from psycopg2 import sql, extras
import json
import requests

In [14]:
# Create Polygon

# Points

pt1 = arcpy.Point(479025.028, 4979084.696) 
pt2 = arcpy.Point(475025.028, 4974084.696) 
pt3 = arcpy.Point(479025.028, 4974084.696)

# Assemble into polygon

poly_array = arcpy.Array([pt1, pt2, pt3, pt1])
poly = arcpy.Polygon(poly_array, spatial_reference=26915)

In [15]:
# Get WKT

wkt = poly.WKT

In [235]:
# Connect to PostGIS Database

pg_connection_dict = {
    'dbname': 'lab0',
    'user': 'postgres',
    'password': 'postgres',
    'port': '5432',
    'host': '35.232.191.92'
}

conn = psycopg2.connect(**pg_connection_dict)

In [236]:
# Check if connected

conn.closed

0

In [237]:
# Check if Database queries work

cur = conn.cursor() # Set up cursor
cur.execute("SELECT 1") # Simple Query - Should return [(1,)]
print(cur.fetchall()) # Print results
cur.close() # Close Cursor

In [239]:
# Example: Add to database

cur = conn.cursor() # Create cursor for commands

# Create example table (Drops example table if exists)

create_cmd = sql.SQL("DROP TABLE IF EXISTS example;CREATE TABLE example (geom geometry);")

cur.execute(create_cmd)

# Insert into table

cur.execute(
    'INSERT INTO example(geom)'
    'VALUES (ST_Transform(ST_SetSRID(ST_GeomFromText(%(geom)s), 26915),4326)::geometry)',
    {'geom': wkt})

conn.commit() # Commit commands
cur.close() # Close cursor

# Close connection

conn.close()

In [286]:
# Get Data back - For Flask App on VM

# Get example 

def get_example():
    
    # Connect to Database
    pg_connection_dict = {
    'dbname': 'lab0',
    'user': 'postgres',
    'password': 'postgres',
    'port': '5432',
    'host': '35.232.191.92'
    }

    conn = psycopg2.connect(**pg_connection_dict)
    
    # Create json cursor
    cur = conn.cursor(cursor_factory = extras.RealDictCursor)
    
    # Get the example
    cmd = """SELECT json_build_object(
    'type', 'FeatureCollection',
    'features', json_agg(ST_AsGeoJSON(example.*)::json)
    )
FROM example;"""
    
    # If want with CRS 'crs', json_build_object('type', 'name', properties', json_build_object( 'name', ST_SRID(geom))::json)::json,
    
    cur.execute(cmd)
#     cur.execute('SELECT ST_ASGeoJSON(geom) FROM example')
    
    conn.commit() # Committ command
    
    geojson = json.loads(json.dumps(cur.fetchall()))[0]["json_build_object"]
    
    # Close connection
    cur.close()
    conn.close()
    
    return geojson

geo = get_example()

In [279]:
# Double check that output

import pprint

pprint.pprint(geo)

{'features': [{'geometry': {'coordinates': [[[[-93.265965808, 44.964891414],
                                              [-93.316438353, 44.919752628],
                                              [-93.265758076, 44.919881808],
                                              [-93.265965808, 44.964891414]]]],
                            'type': 'MultiPolygon'},
               'properties': {},
               'type': 'Feature'}],
 'type': 'FeatureCollection'}


In [282]:
# Looks good, copy that into Flask App on VM
# Need to install psycopg2
# command -> pip install psycopg2-binary

# add all libraries to top of the python script!!

# for the descriptor use both methods ['GET', 'POST']

In [282]:
# Get from flask App - make sure DB and Flask Server Running

r = requests.request('GET', 'http://34.123.205.213:5000/get_data')

geojson = r.json()

# Write to file

with open(r'example.geojson', 'w') as file:
    file.write(str(geojson))
    
# Load into Arcpy

arcpy.conversion.JSONToFeatures('example.geojson', 'example')

# Upload to ArcOnline?

# In ARcGIS Map Viewer Use GeoJSON link